In this notebook, I attempt to use Recurrent Neural Networks (RNNs), Gated Recurrent Units (GRUs) and Long-Short Term Memory (LSTM) to conduct classification

In [2]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader
# from torchvision import datasets, transforms

Reading in the data

In [3]:
train_df = pd.read_csv('data/DL/DL_train.csv')
train_df.head()

,text,humor
0,watch this swimmer disappear into winter storm...,False
1,"they laughed at reagan, too: trump's ideas wil...",False
2,"hey, are you cold? go over to the corner, it i...",True
3,cannot get a standing desk? these are almost a...,False
4,want to hear a joke about my penis? never mind...,True


In [4]:
test_df = pd.read_csv('data/DL/DL_test.csv')
test_df.head()

,text,humor
0,thought up a reddit joke today. when is a tria...,True
1,how much do pirates pay for corn? a buck an ear!,True
2,hillary clinton sent her book to every gop can...,False
3,italian unions lambast new museum boss for wor...,False
4,life below the ocean’s surface wholly depends ...,False


Setting up the hyperparameters

In [5]:
INPUT_SIZE = 784
HIDDEN_SIZE = 300
NUM_CLASSES = 2
NUM_EPOCHS = 2
BATCH_SIZE = 100
LEARNING_RATE = 0.001